# Monitor a Model

When you've deployed a model into production as a service, you'll want to monitor it to track usage and explore the requests it processes. You can use Azure Application Insights to monitor activity for a model service endpoint.

This notebooks is based on the DP-100 tutorial from [here](https://github.com/MicrosoftLearning/mslearn-dp100/blob/main/16%20-%20Monitor%20a%20Model.ipynb)

In [ ]:
## Change working directory
import os

os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import git

repo = git.Repo(os.getcwd(), search_parent_directories=True)
os.chdir(repo.working_tree_dir)

print(os.getcwd())

In [ ]:
# download the model files
import os
from pathlib import Path
from azureml.core import Model, Workspace
workspace = Workspace.from_config()
model_name = 'RuggieroS_Model-EF-MLOPS'
model_version = '4'
output_dir = Path('data/model_files')
model = Model(workspace=workspace,
              name=model_name,
              version=model_version)
output_dir.mkdir(parents=True, exist_ok=True)
model.download(output_dir, exist_ok=True)

print(f'Model files downloaded and saved at location {output_dir}')
model_files_path = os.path.join(output_dir, 'model_pipeline')
list_of_files = os.listdir(model_files_path)
if list_of_files:
    print("The following files are downloaded:")
    for file in list_of_files:
        print(file)
else:
    raise ValueError(
        f"Either the files are not downloaded or the files are not saved at the location {model_files_path}")

In [ ]:
import os

# Create a folder for the deployment files
deployment_folder = './bike_sharing_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_bike_sharing.py'
script_path = os.path.join(deployment_folder,script_file)

Now you need an entry script that the service will use to score new data.

In [ ]:
%%writefile $script_path
import mlflow
import os
import json
import pandas as pd

# Called when the service is loaded
def init():
    global model

    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'),'model_pipeline' )
    mlflow_pipeline = mlflow.pyfunc.load_model(model_path)
    model = mlflow_pipeline._model_impl.python_model.model

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = json.loads(raw_data)['data']
    df_data = pd.DataFrame.from_dict(json.loads(data))
    # Get a prediction from the model
    predictions = model.predict(df_data)
    
    # print the data and predictions (so they'll be logged!)
    log_text = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_text)
    

    return json.dumps({idx: p for idx, p in enumerate(predictions)})

Now you can deploy the service (in this case, as an Azure Container Instance (ACI).

> **Note**: This can take a few minutes - wait until the state is shown as **Healthy**.

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

# Configure the scoring environment
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=Environment.from_conda_specification(
                                       ".venv", "./environment.yml"
                                   ))

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "bikes-service-app-insights"
aci_service = Model.deploy(workspace, service_name, [model], inference_config, deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output = True)
print(aci_service.state)

In [ ]:
aci_service.get_logs()

## Enable Application Insights

Next, you need to enable Application Insights for the service.

In [ ]:
# Enable AppInsights
aci_service.update(enable_app_insights=True)
print('AppInsights enabled!')

## Use the web service

With the service deployed, now you can consume it from a client application.

First, determine the URL to which these applications must submit their requests.

In [ ]:
endpoint = aci_service.scoring_uri
print(endpoint)

Now that you know the endpoint URI, an application can simply make an HTTP request, sending the patient data in JSON (or binary) format, and receive back the predicted class(es).

> **Tip**: If an error occurs because the service endpoint isn't ready. Wait a few seconds and try again!

In [ ]:
# we create some data to test the API.
# Since the preprocessing steps performed in the DataTransformer are not saved in the model, we need to apply the same operations to the data below
import requests
import json

import numpy as np
import pandas as pd


df = pd.read_csv(r'data\BikeSharingPredictionsHours.csv', sep=';')
df.drop(['dteday'], axis=1, inplace=True)
df.drop(['holiday', 'registered', 'casual', 'cnt'], axis=1, inplace=True)
coef = 2 * np.pi / 23.0
df['hour_sin'] = np.sin(coef * df['hr'])
df['hour_cos'] = np.cos(coef * df['hr'])
df.rename(columns={'yr': 'year', 'mnth': 'month', 'weekday': 'week_day',
                   'workingday': 'working_day', 'weathersit': 'weather_situation', 'atemp': 'temp_feel',
                   'hum': 'humidity', 'windspeed': 'wind_speed'}, inplace=True)

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": df.head().to_json()})

# Set the content type
headers = { 'Content-Type':'application/json' }

# Get the predictions
predictions = requests.post(endpoint, input_json, headers = headers)
predictions.json()


Now you can view the data logged for the service endpoint:

1. In the [Azure portal](https://portal.azure.com), open your Machine Learning workspace.
2. On the **Overview** page, click the link for the associated **Application Insights** resource.
3. On the Application Insights blade, click **Logs**. 

    > **Note**: If this is the first time you've opened log analytics, you may need to click **Get Started** to open the query editor. If a tip explaining how to write a query is displayed, close it.

4. Paste the following query into the query editor and click **Run**
    ```
    traces
    |where  message == "STDOUT"
      and customDimensions.["Service Name"] == "bikes-service-app-insights"
    |project timestamp, customDimensions.Content
    ```
5. View the results. At first there may be none, because an ACI web service can take as long as five minutes to send the telemetry to Application Insights. Wait a few minutes and re-run the query until you see the logged data and predictions.
6. When you've reviewed the logged data, close the Application Insights query page.

## Delete the service

When you no longer need your service, you should delete it.

> **Note**: If the service is in use, you may not be able to delete it immediately.

In [ ]:
try:
    aci_service.delete()
    print('Service deleted.')
except Exception as ex:
    print(ex.message)

For more information about using Application Insights to monitor a deployed service, see the [Azure Machine Learning documentation](https://docs.microsoft.com/azure/machine-learning/how-to-enable-app-insights).